### Load Libraries

In [1]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import numpy as np
import time
import cv2
import os
import math

### Mask Detector

In [20]:
def detect_and_predict_mask(frame, faceNet, maskNet):
    # Get height and width of the frame
    (h, w) = frame.shape[:2]
    
    # Preprocess image by getting the blob before face detection using caffe model
    blob = cv2.dnn.blobFromImage(frame, 1.3, (224, 224), (104.0, 177.0, 123.0), False, False)
    
    # Pass blob into the caffe model
    faceNet.setInput(blob)
    
    # Get face detections
    detections = faceNet.forward()
    
    faces = []
    locs = []
    preds = []
    
    # Iterate over all faces detected by the caffe model
    for i in range(0, detections.shape[2]):
        
        # Get the confidence value
        confidence = detections[0, 0, i, 2]
        
        # if confidence passes threshold, it can be processed further
        if confidence > 0.5:
            
            # Get the bounding box of the face in the frame
            box = detections[0,0,i,3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")
            
            # Ensure that bounding box still lies inside the frame
            (startX, startY) = (max(0, startX), max(0, startY))
            (endX, endY) = (min(w-1, endX), min(h-1, endY))
            
            # Crop image to only the face
            face = frame[startY:endY, startX:endX]
            
            # Convert face into RGB format before fitting it into the model
            face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
            
            # Resize image to the standard size
            face = cv2.resize(face, (224,224))
            
            # convert image into an array
            face = img_to_array(face)
            
            # Preprocess image to meet the format of MobileNet
            face = preprocess_input(face)
            
            # Append filtered face into a list
            faces.append(face)
            
            # Append the bounding box into a list
            locs.append((startX, startY, endX, endY))
            
    if len(faces) > 0:
        # Convert face into a numpy array
        faces = np.array(faces, dtype="float32")
        
        # Predict the face using the loaded Mask Detector
        preds = maskNet.predict(faces, batch_size=32)
        
    return (locs, preds)

### Face Recognition

In [3]:
def detect_and_recognize(frame, model, rect):
    name = ""
    conf = -1
    
    # Get the bounding box obtained from the mask detector preprocessing
    (startX, startY, endX, endY) = rect
    
    # Crop frame to only the bounding box
    gray_cropped = frame[startY:endY, startX:endX]
    
    # Convert cropped face into grayscale
    gray_cropped = cv2.cvtColor(gray_cropped, cv2.COLOR_BGR2GRAY)
    
    try:
        # Use the trained LBPH Face Recognizer to identify the subject in the frame
        predicted, conf = model.predict(gray_cropped)
        
        # Transform prediction into its corresponding name
        if(predicted == 0):
            name =  "Chaeyoung"
        elif(predicted == 1):
            name = "Dahyun"
        elif(predicted == 2):
            name = "Dave"
        elif(predicted == 3):
            name = "Jeongyeon"
        elif(predicted == 4):
            name = "Jihyo"
        elif(predicted == 5):
            name = "Mina"
        elif(predicted == 6):
            name = "Momo"
        elif(predicted == 7):
            name = "Nayeon"
        elif(predicted == 8):
            name = "Sana"
        elif(predicted == 9):
            name = "Tzuyu"

    except:
        print('No Faces found')
        pass
            
    return name, conf

### Load the Caffe Model for Face Detection

In [4]:
prototxtPath = r"face_detector\deploy.prototxt.txt"
weightsPath = r"face_detector\res10_300x300_ssd_iter_140000.caffemodel"

faceNet = cv2.dnn.readNetFromCaffe(prototxtPath, weightsPath)

### Load the trained Mask Detector

In [5]:
maskNet = load_model("mask_detector")

### Load the trained LBPH Face Recognizer

In [6]:
lbph = cv2.face.LBPHFaceRecognizer_create()
model = lbph.read('TrainedLBPHFaceRecognizer')

### Main

In [24]:
# Start WebCam
cap = cv2.VideoCapture(0)

count = -1
status = 1

while True:
    # Get Frame
    ret, frame = cap.read()
    
    # Detect and predict mask using the trained MobileNet Mask Detector
    (locs, preds) = detect_and_predict_mask(frame, faceNet, maskNet)
    
    # Iterate over all predictions found in the frame
    for (box, pred) in zip(locs, preds):
        
        # Get bounding box
        (startX, startY, endX, endY) = box
        
        # Get mask prediction outcome
        (mask, withoutMask) = pred
    
        # Identify subject in the frame using the trained LBPH Face Recognizer
        res_name, res_conf = detect_and_recognize(frame, lbph, box)
        
        # If confidence level is under 70 it passes the threshold and prediction is valid
        if res_conf < 70:
            
            # Print prediction result and its corresponding confidence level
            print("{}: {:}".format(res_name, math.floor((100 - res_conf)*10)/10))
            
            # If probability of mask greater than without mask, then subject is wearing a mask
            if mask > withoutMask:
                label = "Thanks for wearing a mask, {}!".format(res_name)
                color = (0,255,0)
            # Otherwise, subject is not wearing a mask
            else:
                label = "Please wear a mask, {}!".format(res_name)
                color = (0,0,255)
            
            # Write the text that includes the name of the subject, and
            # whether or not subject is wearing a mask
            cv2.putText(frame, label, (15,35),
                       cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
            
            # Annotate the bounding box of the subject in the frame
            cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)
    
    # Display the frame
    cv2.imshow("Frame", frame)
    
    # Define quit key
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

Dave: 32.6
Dave: 30.7
Dave: 32.4
Dave: 30.7
Dave: 30.9
Dave: 31.4
Dave: 31.2
Dave: 30.7
Dave: 30.8
Dave: 31.5
Dave: 32.3
Dave: 33.3
Dave: 31.2
Dave: 31.5
Dave: 33.7
Dave: 33.2
Dave: 31.4
Dave: 31.7
Dave: 31.2
Dave: 31.7
Dave: 31.4
Dave: 30.2
Dave: 34.1
Dave: 31.7
Dave: 31.8
Dave: 31.0
Dave: 31.3
Dave: 31.3
Dave: 31.8
Dave: 33.6
Dave: 33.9
Dave: 32.4
Dave: 33.4
Dave: 33.1
Dave: 30.1
Dave: 30.1
Dave: 30.2
Dave: 30.0
Dave: 30.5
Dave: 31.1
Dave: 30.0
Dave: 31.2
Dave: 32.4
Dave: 32.7
Dave: 32.8
Dave: 31.5
Dave: 31.2
Dave: 33.1
Dave: 32.5
Dave: 32.5
Dave: 32.1
Dave: 31.2
Dave: 32.5
Dave: 33.8
Dave: 32.1
Dave: 32.0
Dave: 31.9
Dave: 30.0
Dave: 30.8
Dave: 33.9
Dave: 35.3
Dave: 32.4
Dave: 30.6
Dave: 30.5
Dave: 31.2
Dave: 30.0
Dave: 32.6
Dave: 32.0
Dave: 31.7
Dave: 31.6
Dave: 32.5
Dave: 30.6
Dave: 31.4
Dave: 31.2
Dave: 31.3
Dave: 30.7
Dave: 31.5
Dave: 31.7
Dave: 30.5
Dave: 31.0
Dave: 31.8
Dave: 32.0
Dave: 31.0
Dave: 30.0
Dave: 30.9
Dave: 31.8
Dave: 32.2
Dave: 31.5
Dave: 31.9
Dave: 31.5
Dave: 32.6